# Job Search Tracker 
The purpose of this notebook is to create a database of jobs I have applied to and are still active as well as track the jobs I have been rejected from. I made this because I wanted to better filter out jobs that I had been rejected from but not lose the job entry entirely while also focusing on active job postings and applications. With this I hope to see what similarities applications may have so that I can adjust my resume and cover letters accordingly as well as ensure I keep consistant contact with these job applications with the overall goal of securing a position in the data analyst/data science/data engineering field. 

### Library Imports and Adapter/Converter definiton for dates

In [2]:
%pip install ipywidgets pandasgui
import sqlite3
import datetime
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
from pandasgui import show


warnings.filterwarnings("ignore", category=DeprecationWarning)
# Register custom adapters and converters for datetime objects
def adapt_datetime(dt):
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def convert_datetime(s):
    return datetime.strptime(s.decode(), "%Y-%m-%d %H:%M:%S")

sqlite3.register_adapter(datetime, adapt_datetime)
sqlite3.register_converter("timestamp", convert_datetime)

output = widgets.Output()


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/983.9 kB ? eta -:--:--
     ------------------------------------- 983.9/983.9 kB 45.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): fi

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Functions

#### SQL Create Tables Jobs and Rejected Jobs

In [209]:
def create_tables():
    conn = sqlite3.connect('job_search.db')
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Jobs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        job_title TEXT NOT NULL,
        company TEXT NOT NULL,
        status TEXT DEFAULT 'Need to apply',
        job_found_location TEXT,
        url TEXT,
        app_due_date TEXT,
        contact_person TEXT,
        contact_title TEXT,
        contact TEXT,
        location TEXT,
        hours TEXT,
        skills_req_to_highlight TEXT,
        notes_about_company TEXT,
        date_applied DATE,
        days_since_app_submitted INTEGER, 
        date_of_confirmation_reply DATE,
        days_from_app_submitted_to_confirmation_reply INTEGER,
        days_since_confirmation_reply INTEGER
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Rejected_Jobs (
        id INTEGER PRIMARY KEY,
        job_title TEXT NOT NULL,
        company TEXT NOT NULL,
        status TEXT DEFAULT 'Rejected',
        job_found_location TEXT,
        url TEXT,
        app_due_date TEXT,
        contact_person TEXT,
        contact_title TEXT,
        contact TEXT,
        location TEXT,
        hours TEXT,
        skills_req_to_highlight TEXT,
        notes_about_company TEXT,
        date_applied DATE,
        date_confirmation_reply TEXT,
        days_from_app_submitted_to_confirmation_reply INTEGER,
        date_rejected DATE,
        days_between_submission_and_rejection INTEGER,
        days_between_confirmation_and_rejection INTEGER
    )
    ''')

    conn.commit()
    conn.close()
    

#### Add job function

In [210]:
def add_job():
    job_title_user_input = widgets.Text(description="Job Title:", style={'description_width': 'initial'})
    company_name_user_input = widgets.Text(description="Company Name:", style={'description_width': 'initial'})
    status_user_input = widgets.Dropdown(
        options=["Need to apply", "Applied", "Followed Up", "Rejected", "Interview process started", "Landed"],
        description="Status:", style={'description_width': 'initial'}
    )
    job_found_location_user_input = widgets.Text(description="Job Found Location:", style={'description_width': 'initial'})
    url_user_input = widgets.Text(description="URL:", style={'description_width': 'initial'})
    app_due_date_user_input = widgets.Text(description="Application Due Date:", style={'description_width': 'initial'})
    contact_person_user_input = widgets.Text(description="Contact Person:", style={'description_width': 'initial'})
    contact_title_user_input = widgets.Text(description="Contact Title:", style={'description_width': 'initial'})
    contact_user_input = widgets.Text(description="Contact:", style={'description_width': 'initial'})
    location_user_input = widgets.Text(description="Location:", style={'description_width': 'initial'})
    hours_user_input = widgets.Text(description="Hours:", style={'description_width': 'initial'})
    skills_req_to_highlight_user_input = widgets.Text(description="Skills Required to Highlight:", style={'description_width': 'initial'})
    notes_about_company_user_input = widgets.Text(description="Notes About Company:", style={'description_width': 'initial'})
    date_applied_user_input = widgets.Text(description="Date Applied:", style={'description_width': 'initial'})
    date_of_confirmation_reply_user_input = widgets.Text(description="Date of Confirmation/Reply:", style={'description_width': 'initial'})
    
    submit_button = widgets.Button(description="Submit", style={'description_width': 'initial'})   
    
    def on_submit_button_clicked(b):
        job_title = job_title_user_input.value
        company = company_name_user_input.value
        status = status_user_input.value
        job_found_location = job_found_location_user_input.value
        url = url_user_input.value
        app_due_date = app_due_date_user_input.value
        contact_person = contact_person_user_input.value
        contact_title = contact_title_user_input.value
        contact = contact_user_input.value
        location = location_user_input.value
        hours = hours_user_input.value
        skills_req_to_highlight = skills_req_to_highlight_user_input.value
        notes_about_company = notes_about_company_user_input.value
        date_applied = date_applied_user_input.value
        date_of_confirmation_reply = date_of_confirmation_reply_user_input.value
    
        conn = sqlite3.connect('job_search.db')
        cursor = conn.cursor()
        
        # Calculate days_from_app_submitted_to_confirmation_reply
        if date_applied and date_of_confirmation_reply:
            date_applied = datetime.strptime(date_applied, "%m/%d/%Y").date()
            date_of_confirmation_reply = datetime.strptime(date_of_confirmation_reply, "%m/%d/%Y").date()
            days_from_app_submitted_to_confirmation_reply = (date_of_confirmation_reply - date_applied).days
            days_since_app_submitted = (datetime.now() - date_applied).days
            days_since_confirmation_reply = (datetime.now() - date_of_confirmation_reply).days


        elif date_applied and not date_of_confirmation_reply:
            days_from_app_submitted_to_confirmation_reply = None
            date_applied = datetime.strptime(date_applied, "%m/%d/%Y").date()
            date_of_confirmation_reply = None
            days_since_app_submitted = (datetime.now() - date_applied).days
            days_since_confirmation_reply = None
            
        else:
            days_from_app_submitted_to_confirmation_reply = None
            date_applied = None
            date_of_confirmation_reply = None
            days_since_app_submitted = None
            days_since_confirmation_reply = None

        cursor.execute('''
        INSERT INTO Jobs 
        (job_title, company, status, job_found_location, url, 
        app_due_date, contact_person, contact_title, contact, location, 
        hours, skills_req_to_highlight, notes_about_company, date_applied, days_since_app_submitted, 
        date_of_confirmation_reply, days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply)
        VALUES (?, ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, 
                ?, ?, ?)
        ''', (job_title, company, status, job_found_location, url, 
              app_due_date, contact_person, contact_title, contact, location, 
              hours, skills_req_to_highlight, notes_about_company, date_applied, days_since_app_submitted, 
              date_of_confirmation_reply, days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply))

        conn.commit()
        conn.close()
        with output:
            print("Job added successfully!")
            output.clear_output(wait=True)
    submit_button.on_click(on_submit_button_clicked)
    
    with output:
        output.clear_output()
        display(job_title_user_input, company_name_user_input, status_user_input, 
                job_found_location_user_input, url_user_input, app_due_date_user_input, 
                contact_person_user_input, contact_title_user_input, contact_user_input, 
                location_user_input, hours_user_input, skills_req_to_highlight_user_input, 
                notes_about_company_user_input, date_applied_user_input, date_of_confirmation_reply_user_input, 
                submit_button)
        

#### Update Job Status Function
If the job status is changed from "Need to Apply" to "Applied" the user is asked to enter the date when the application was submitted in MM/DD/YYYY format. This function will also calculate how many days since the application was submitted based on the current date to account for jobs that are being updated after time has passed. 
To change the date of the application confirmation or reply, the update_reply function must be called. 
To view the jobs table, the View Active jobs function needs to be called.

If the job status is changed to "Rejected" the user is asked to enter the date of rejection in MM/DD/YYYY format. Then the number of days from the application submission to rejection is calculated and if there is a date of confirmation/reply, the number of days from last reply to rejection is also calculated. If there is no date of confirmation/reply then this value is empty. 

Then that job is added to the Rejected_Jobs table with columns: "[id, job_title, company, status, job_found_location, url, app_due_date, contact_person, contact_title, contact, location, hours, skills_req_to_highlight, notes_about_company, date_applied, date_confirmation_reply, date_rejected, days_between_submission_and_rejection, and days_from_confirmation_reply_to_rejection]" 

For other status options there is no change to the job besides the status. This is a place of exploration in the future. For example, find the number of days since last follow up and who was contacted or days from application submission to when interview process is started. Also add entry to detail interview process. 

In [211]:
def update_job_status():
    conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()
    cursor.execute('SELECT id, job_title, company, status FROM Jobs')
    jobs = cursor.fetchall()
    conn.close()
    
    df = pd.DataFrame(jobs, columns=['Job ID', 'Job Title', 'Company', 'Status'])
    scrollable_output = widgets.HTML(
    value=f"""
    <div style="width: initial; overflow-x: auto; overflow-y: auto; border: 2px solid black;">
        <style> 
            table {{
                width: 100%;
                table-layout: auto;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid black;
                padding: 8px;
                text-align: center;
            }}
        </style>
        {df.to_html(index=False, classes='table table-striped')}
    </div>
    """
    )
    
    output.clear_output(wait=True)
    display(scrollable_output)
    
    job_id_input = widgets.IntText(description="Job ID:", style={'description_width': 'initial'})
    new_status_input = widgets.Dropdown(
        options=["Need to apply", "Applied", "Followed Up", "Rejected", "Interview process started", "Landed"],
        description="New Status:", style={'description_width': 'initial'}
    )
    date_applied_input = widgets.Text(description="Date Applied (MM/DD/YYYY):", style={'description_width': 'initial'})
    date_rejected_input = widgets.Text(description="Date Rejected (MM/DD/YYYY):", style={'description_width': 'initial'})
    confirmation_input = widgets.Text(description="Are you sure you want to assign this job as Rejected: (Y/N):", style={'description_width': 'initial'})
    date_of_confirmation_reply_input = widgets.Text(description="Date of Confirmation/Reply (MM/DD/YYYY):", style={'description_width': 'initial'})
    submit_button = widgets.Button(description="Submit", style={'description_width': 'initial'})
    def on_submit_button_clicked(b):
        job_id = job_id_input.value
        new_status = new_status_input.value
        confirmation = confirmation_input.value
        
        conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)
        cursor = conn.cursor()
        cursor.execute('SELECT id FROM Jobs WHERE id = ?', (job_id,))
        job_id_exists = cursor.fetchone()
        if not job_id_exists:
            with output:
                print(f"Job ID {job_id} does not exist. Please enter a valid Job ID.")
                output.clear_output(wait=True)
                display(job_id_input, new_status_input, submit_button)
            conn.close()
            return
            
        if new_status == "Applied":
            try:
                date_applied = date_applied_input.value
                date_applied_dt = datetime.strptime(date_applied, "%m/%d/%Y").date()
                days_since_app_submitted = (datetime.now().date() - date_applied_dt).days

                cursor.execute('''UPDATE Jobs 
                            SET status = ?,  date_applied = ?, days_since_app_submitted = ?
                            WHERE id = ?''', (new_status, date_applied_dt, days_since_app_submitted, job_id))
                print("Job Application date updated successfully.")
            except ValueError:
                with output:
                    print("Invalid date format. Please enter the date in MM/DD/YYYY format.") 
                    output.clear_output(wait=True)
                    display(date_applied_input, submit_button)
                return  
    
        elif new_status == "Rejected":
            if confirmation.lower in ['y', 'yes']:
                try:
                    date_rejected = date_rejected_input.value
                    date_rejected_dt = datetime.strptime(date_rejected, "%m/%d/%Y").date()
                except ValueError:
                    with output:
                        print("Invalid date format. Please enter the date in MM/DD/YYYY format.")
                        display(date_rejected_input, submit_button)
                        output.clear_output(wait=True)
                    return    
                cursor.execute('''SELECT id, job_title, company, status, job_found_location, url, app_due_date, 
                                contact_person, contact_title, contact, location, hours, skills_req_to_highlight, 
                                notes_about_company, date_applied, date_confirmation_reply, 
                                FROM Jobs WHERE id = ?''', (job_id,))
                job = cursor.fetchone()
                    
                date_applied = job[14]
                date_of_confirmation_reply = job[15]
                    
                    
                if date_applied:
                    try:
                        date_applied_dt = datetime.strptime(date_applied, "%m/%d/%Y").date
                        days_between_submission_and_rejection = (date_rejected_dt - date_applied_dt).days
                    except ValueError:
                        with output:
                            print("Invalid date format for date applied. Please enter the date in MM/DD/YYYY format.")
                            display(date_applied_input, submit_button)
                            output.clear_output(wait=True)
                        return
                else:
                    with output:
                        print("No application submission date found. Please enter the date the application was submitted.")
                        display(date_applied_input, submit_button)
                        output.clear_output(wait=True)
                    return
                        
                if date_of_confirmation_reply:
                    try:
                        date_of_confirmation_reply_dt = datetime.strptime(date_of_confirmation_reply, "%m/%d/%Y").date()
                        days_from_confirmation_reply_to_rejection = (date_rejected_dt - date_of_confirmation_reply_dt).days
                    except ValueError:
                        with output:
                            print("Invalid date format for date of confirmation/reply. Please enter the date in MM/DD/YYYY format.")
                            display(date_of_confirmation_reply_input, submit_button)
                            output.clear_output(wait=True)
                        return
                else:
                    days_from_confirmation_reply_to_rejection = None

                cursor.execute('''
                INSERT INTO Rejected_Jobs (id, job_title, company, status, job_found_location, url, app_due_date,
                                        contact_person, contact_title, contact, location, hours, 
                                        skills_req_to_highlight, notes_about_company, date_applied, 
                                        date_confirmation_reply, date_rejected, 
                                        days_between_submission_and_rejection, 
                                        days_from_confirmation_reply_to_rejection)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (*job, date_rejected, days_between_submission_and_rejection, days_from_confirmation_reply_to_rejection))
                cursor.execute('DELETE FROM Jobs WHERE id = ?', (job_id,))
                print("Job details updated successfully.")   
                
            elif confirmation.lower() in ['n', 'no']:
                with output:
                    print("Job not changed to Rejected. Please select a different status.")
                    display(job_id_input, new_status_input, confirmation_input, submit_button)
                    output.clear_output(wait=True)
                return
            else:
                with output:
                    print("Invalid Confirmation. Please enter 'Y' or 'N'.")
                    display(confirmation_input, submit_button)
                    output.clear_output(wait=True)
                return
        else:
            cursor.execute('UPDATE Jobs SET status = ? WHERE id = ?', (new_status, job_id))

        conn.commit()
        conn.close()
        with output:
            print("Job status updated successfully!")
            output.clear_output(wait=True)
    
    submit_button.on_click(on_submit_button_clicked)
    with output:
        display(job_id_input, new_status_input, submit_button)
        output.clear_output(wait=True)
        

#### Update Reply/Confirmation Date function
This function updates a job applications last contact. Since some jobs will send a application submission confirmation email and some don't I wanted to create something that could track that. Also for jobs where there is a dialog conversation, I want to track the date of last contact so that I don't lose contact or be overbearing in my contact. 

This function takes a date of confirmation/reply and job id as input and fills the table with that date as well as the number of days from application submission to the last contact and the number of days since that last contact based on the current date when the program is run.  

In [212]:
def update_reply_date():
    conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()
    cursor.execute('''SELECT id, job_title, company, status, date_applied, days_since_app_submitted, 
                   date_of_confirmation_reply, days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply 
                   FROM Jobs''')
    jobs = cursor.fetchall()
    conn.close()
    
    df = pd.DataFrame(jobs, columns=['Job ID', 'Job Title', 'Company', 'Status', 'Date Applied', 'Days since app submitted', "Date of conf/reply",'days from app submitted to conf/reply', 'Days since conf/reply'])
    scrollable_output = widgets.HTML(
    value=f"""
    <div style="width: initial; overflow-x: auto; overflow-y: auto; border: 2px solid black;">
        <style> 
            table {{
                width: 100%;
                table-layout: auto;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid black;
                padding: 8px;
                text-align: center;
            }}
        </style>
        {df.to_html(index=False, classes='table table-striped')}
    </div>
    """
    )
    
    output.clear_output(wait=True)
    display(scrollable_output)

    
    job_id_input = widgets.IntText(description="Job ID:", style={'description_width': 'initial'})
    date_of_confirmation_reply_input = widgets.Text(description="Date of Confirmation/Reply (MM/DD/YYYY):", style={'description_width': 'initial'})
    date_applied_input = widgets.Text(description="No application submission date! Please enter Date Applied (MM/DD/YYYY):", style={'description_width': 'initial'})
    submit_button = widgets.Button(description="Submit", style={'description_width': 'initial'})
    
    def on_submit_button_clicked(b):
        job_id = job_id_input.value
        date_of_confirmation_reply = date_of_confirmation_reply_input.value
        
        conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)
        cursor = conn.cursor()
        cursor.execute('''SELECT id, date_applied, date_of_confirmation_reply, 
                    days_since_confirmation_reply 
                    FROM Jobs 
                    WHERE id = ?''', (job_id,))
        job = cursor.fetchone()

        
        date_applied = job[1]
    
        if date_applied:
            try:
                date_applied_dt = datetime.strptime(date_applied, "%m/%d/%Y").date()
                try:
                    date_of_confirmation_reply_dt = datetime.strptime(date_of_confirmation_reply, "%m/%d/%Y").date()
                    days_from_app_submitted_to_confirmation_reply = (date_of_confirmation_reply_dt - date_applied_dt).days
                    days_since_confirmation_reply = (datetime.now().date() - date_of_confirmation_reply_dt).days
                    
                    cursor.execute('''UPDATE Jobs
                                   SET date_of_confirmation_reply = ?, 
                                        days_from_app_submitted_to_confirmation_reply = ?, 
                                        days_since_confirmation_reply = ?
                                    WHERE id = ?''', (date_of_confirmation_reply_dt, days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply, job_id))
                    conn.commit()
                    print("Job reply date updated successfully.")
                except ValueError:
                    with output:
                        print("Invalid date format for date of confirmation/reply. Please enter the date in MM/DD/YYYY format.")
                        display(date_of_confirmation_reply_input, submit_button)
                        output.clear_output(wait=True)
                    return
            except ValueError:
                with output:
                    print("Invalid date format for date applied. Please enter the date in MM/DD/YYYY format.")
                    display(date_applied_input, submit_button)
                    output.clear_output(wait=True)
                return
        else:
            with output:
                print("No application submission date found. Please enter the date the application was submitted in MM/DD/YYYY format.")
                display(date_applied_input, submit_button)
                output.clear_output(wait=True)
            return
        conn.close()
    
    submit_button.on_click(on_submit_button_clicked)
    with output:
        display(job_id_input, date_of_confirmation_reply_input, submit_button)
        output.clear_output(wait=True)


#### Update Other Function
This function is added to allow for other edits if the user made a mistake or found more information in any of their inputs that is not the application submission date or application confirmation/reply date. This function goes through each column of a specifed job ID, displays the column name and the current recorded entry. It asks the user to enter a new value or to press Enter to keep the current value. The columns the function iterates through are "["job_title", "company", "job_found_location", "app_due_date", "contact_person", "contact_title", "contact", "location", "hours", "skills_req_to_highlight", "notes_about_company"]"

In [213]:
def update_other():
    conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()
    cursor.execute('''SELECT id, job_title, company, status, date_applied, days_since_app_submitted, 
                   date_of_confirmation_reply, days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply 
                   FROM Jobs''')
    jobs = cursor.fetchall()
    conn.close()
    
    df_other = pd.DataFrame(jobs, columns=['Job ID', 'Job Title', 'Company', 'Status', 'Date Applied', 
                                           'Days since app submitted', "Date of conf/reply",'days from app submitted to conf/reply', 
                                           'Days since conf/reply'])
    scrollable_output = widgets.HTML(
    value=f"""
    <div style="width: initial; overflow-x: auto; overflow-y: auto; border: 2px solid black;">
        <style> 
            table {{
                width: 100%;
                table-layout: auto;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid black;
                padding: 8px;
                text-align: center;
            }}
        </style>
        {df_other.to_html(index=False, classes='table table-striped')}
    </div>
    """
    )
    output.clear_output(wait=True)
    display(scrollable_output)
    
    job_id_input = widgets.IntText(description="Job ID to edit:", style={'description_width': 'initial'})
    submit_button = widgets.Button(description="Submit", style={'description_width': 'initial'})
    
    def on_submit_button_clicked(b):
        job_id = job_id_input.value
        conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)

        cursor = conn.cursor()
        cursor.execute('''SELECT id, job_title, company, job_found_location, app_due_date, 
                   contact_person, contact_title, contact, 
                   location, hours, skills_req_to_highlight, notes_about_company 
                   FROM Jobs 
                   WHERE id = ?''', (job_id,))
        job = cursor.fetchone()

        if not job:
            with output:
                print("Job ID not found.")
                output.clear_output(wait=True)
            conn.close()
            return

    # Column names in the Jobs table
        columns = ["job_title", "company", "job_found_location", "app_due_date", "contact_person", "contact_title", "contact", "location", "hours", "skills_req_to_highlight", "notes_about_company"]

    # Display current values and prompt for new values
        input_widgets = []
        for i, column in enumerate(columns):
            current_value = job[i+1]
            new_input_widgets = widgets.Text(value=str(current_value), description=f"{column}:", style={'description_width': 'initial'})
            input_widgets.append(new_input_widgets)
        
        update_button = widgets.Button(description="Update", style={'description_width': 'initial'})
        
        def on_update_button_clicked(b):
            updated_values = []
            for i, widget in enumerate(input_widgets):
                new_value = widget.value.strip()
                if new_value == "":
                    updated_values.append(job[i+1])
                else:
                    updated_values.append(new_value)

            # Update the database
            update_query = f'''
            UPDATE Jobs
            SET {", ".join([f"{column} = ?" for column in columns])}
            WHERE id = ?
            '''
            cursor.execute(update_query, (*updated_values, job_id))

            conn.commit()
            conn.close()
            with output:
                output.clear_output()
                print("Job updated successfully!")
                output.clear_output(wait=True)
        update_button.on_click(on_update_button_clicked)
        with output:
            output.clear_output(wait=True)
            display(widgets.VBox(input_widgets + [update_button]))
            
        
    submit_button.on_click(on_submit_button_clicked)
    
    with output:
        display(job_id_input, submit_button)
        output.clear_output(wait=True)



#### View Active Jobs Function
This function displays the current Jobs table. Running this will update the number of days type calculators to use the current date. 

In [214]:
def view_active_jobs():
    conn = sqlite3.connect('job_search.db', detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()
    cursor.execute(''' SELECT * 
                   FROM Jobs''')
    job_all = cursor.fetchall()
    
    current_date = datetime.now().date()
    
    def update_jobs(job):
        (job_id, job_title, company, status, job_found_location, url, app_due_date, contact_person, contact_title,
        contact, location, hours, skills_req_to_highlight, notes_about_company,
        date_applied, days_since_app_submitted, date_of_confirmation_reply, 
        days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply) = job
        
        if date_applied and isinstance(date_applied, str):
            date_applied = datetime.strptime(date_applied, "%Y-%m-%d").date()
        if date_of_confirmation_reply and isinstance(date_of_confirmation_reply, str):
            date_of_confirmation_reply = datetime.strptime(date_of_confirmation_reply, "%Y-%m-%d").date()
        
        if status != "Need to apply":
            if not date_applied:
                with output:    
                    print(f"No application submission date found for job ID {job_id} Title ({job_title} at company {company}). Date of application submission is required.")
                    date_applied_input = widgets.Text(description="Please enter Date Applied (MM/DD/YYYY):", style={'description_width': 'initial'})
                    submit_button = widgets.Button(description="Submit", style={'description_width': 'initial'})
                    output.clear_output(wait=True)
                    
                    def on_submit_button_clicked(b):
                        try: 
                            date_applied = date_applied_input.value
                            date_applied_dt = datetime.strptime(date_applied, "%m/%d/%Y").date()
                            days_since_app_submitted = (current_date - date_applied_dt).days
                            if date_of_confirmation_reply:
                                days_since_confirmation_reply = (current_date - date_of_confirmation_reply).days   
                                days_from_app_submitted_to_confirmation_reply = (date_of_confirmation_reply - date_applied_dt).days
                            else:
                                days_since_confirmation_reply = None
                                days_from_app_submitted_to_confirmation_reply = None
                            
                            cursor.execute('''UPDATE Jobs
                                             SET date_applied = ?, days_since_app_submitted = ?, date_of_confirmation_reply = ?,
                                                days_from_app_submitted_to_confirmation_reply = ?, days_since_confirmation_reply = ?
                                             WHERE id = ?''', (date_applied_dt, days_since_app_submitted, date_of_confirmation_reply,
                                                               days_from_app_submitted_to_confirmation_reply, 
                                                               days_since_confirmation_reply, job_id))
                            conn.commit()
                            with output:
                                print(f"Job ID {job_id} Title ({job_title} at {company}) updated successfully.")
                                output.clear_output(wait=True)
                        except ValueError:
                            with output:
                                print("Invalid date format. Please enter the date in MM/DD/YYYY format.")
                                display(date_applied_input, submit_button)
                                output.clear_output(wait=True)
                    submit_button.on_click(on_submit_button_clicked)
                    display(date_applied_input, submit_button)
                    return date_applied_input, submit_button, job
            else:
                days_since_app_submitted = (current_date - date_applied).days
                if date_of_confirmation_reply:
                    days_since_confirmation_reply = (current_date - date_of_confirmation_reply).days
                    days_from_app_submitted_to_confirmation_reply = (date_of_confirmation_reply - date_applied).days
                else:
                    days_since_confirmation_reply = None
                    days_from_app_submitted_to_confirmation_reply = None
        else:
            date_applied = None
            date_of_confirmation_reply = None
            days_since_app_submitted = None
            days_since_confirmation_reply = None
            days_from_app_submitted_to_confirmation_reply = None
        
        cursor.execute('''UPDATE Jobs
                          SET date_applied = ?, days_since_app_submitted = ?, date_of_confirmation_reply = ?,
                              days_from_app_submitted_to_confirmation_reply = ?, days_since_confirmation_reply = ?
                          WHERE id = ?''', (date_applied.strftime("%Y-%m-%d") if date_applied else None, days_since_app_submitted, date_of_confirmation_reply, 
                                            days_from_app_submitted_to_confirmation_reply, days_since_confirmation_reply,
                                            job_id))
        conn.commit()
    
    for job in job_all:
        result = update_jobs(job)
        if result:
            date_applied_input, submit_button = result
            return
    
    df_active_jobs = pd.read_sql_query('''SELECT * From Jobs ''', conn)
    scrollable_output = widgets.HTML(
    value=f"""
    <div style="width: 1135px; height: 500px; overflow-x: auto; overflow-y: auto; border: 2px solid black;">
        <style> 
            table {{
                table-layout: auto;
                border-collapse: collapse;
            }}
            th, td {{
                border: 1px solid black;
                padding: 5px;
                text-align: center;
            }}
        </style>
        {df_active_jobs.to_html(index=False, classes='table table-striped')}
    </div>
    """
    )
    
    output.clear_output(wait=True)
    display(scrollable_output)
    conn.close()


#### Exit Menu Function
This function exits the job tracker menu. To make any changes to the database the program must be run again.

In [215]:
def exit_menu():
    clear_output(wait=True)
    exit_options = widgets.Dropdown(
        options=["Clear all tables", "Keep active jobs table"],
        description='Select an exit option:', style={'description_width': 'initial'}
    )
    
    submit_button = widgets.Button(description="Submit", style={'description_width': 'initial'})
    def on_submit_button_clicked(b):
        choice = exit_options.value
        if choice == "Clear all tables":
            clear_output(wait=True)
            print("All tables cleared. Exiting menu.")
        elif choice == "Keep active jobs table":
            clear_output(wait=True)
            print("Keeping active jobs table displayed. Exiting menu.")
            view_active_jobs()
        print("To make any changes or updates to database, please re-run the program.")
    
    submit_button.on_click(on_submit_button_clicked)
    display(exit_options, submit_button)


#### Job Tracker Menu Function
This function provides the Job tracker menu where the user selects the action they want to perform. This function has all of the function calls of the preceeding function defintions depending on the users selection. It is given to the user via dropdown menu. This function also creates the tables Jobs and Rejected_Jobs. 

In [216]:
def on_menu_change(change):
    output.clear_output(wait=True)
    with output:
        if change['new'] == 0:
            print("Please select an option from the dropdown menu.")
        elif change['new'] == 1:
            print("Add job selected")
            add_job()
        elif change['new'] == 2:
            print("Update job status selected")
            update_job_status()
        elif change['new'] == 3:
            print("Update reply date selected")
            update_reply_date()
        elif change['new'] == 4:
            print("Update other selected")
            update_other()
        elif change['new'] == 5:
            print("View active jobs selected")
            view_active_jobs()
        elif change['new'] == 6:
            print("Exit selected")
            exit_menu()
            
job_tracker_menu_options = widgets.Dropdown(
    options=[
        ("Job Tracker Menu", 0),
        ("Add job", 1),
        ("Update job status", 2),
        ("Update reply date", 3),
        ("Update other", 4),
        ("View active jobs", 5),
        ("Exit", 6)
    ],
    value=0,
    description='Select an option:',
    style={'description_width': 'initial'}
)
create_tables()
job_tracker_menu_options.observe(on_menu_change, names='value')
display(job_tracker_menu_options, output)
    

Dropdown(description='Select an option:', options=(('Job Tracker Menu', 0), ('Add job', 1), ('Update job statu…

Output()